#Without Threading

In [5]:
import requests
import time

In [6]:
url = "https://raw.githubusercontent.com/Monalsingh/VideoBroadcaster/refs/heads/main/static/default-office-animated.png"

In [7]:
def download_file(process_name, url, file_path):
    try:
        print(f"Download process name started : {process_name}")
        response = requests.get(url)
        with open(file_path, "wb") as file:
            for chunk in response.iter_content(chunk_size=8192):
                if chunk:
                    file.write(chunk)
        print("File downloaded successfully")
    except Exception as e :
        print(f"Error downloading file : {e}")
    print(f"Process name completed : {process_name}")

In [8]:
t1 = time.time()
download_file("Download without thread 1", url, "a.png")
download_file("Download without thread 2", url, "b.png")
download_file("Download without thread 3", url, "c.png")
t2 = time.time()
print(f"Time taken(seconds) : {t2-t1}")

Download process name started : Download without thread 1
File downloaded successfully
Process name completed : Download without thread 1
Download process name started : Download without thread 2
File downloaded successfully
Process name completed : Download without thread 2
Download process name started : Download without thread 3
File downloaded successfully
Process name completed : Download without thread 3
Time taken(seconds) : 0.840712308883667


In [10]:
!ls

a.png  b.png  c.png  sample_data


#Threading

In [13]:
import threading

In [18]:
t1 = threading.Thread(target=download_file, args=("Download with thread 1", url, "a1.png"))
t2 = threading.Thread(target=download_file, args=("Download with thread 2", url, "b1.png"))
t3 = threading.Thread(target=download_file, args=("Download with thread 3", url, "c1.png"))

### thread.start()

In [15]:
t1_t = time.time()
t1.start()
t2.start()
t3.start()
print("Main program done!!")
t2_t = time.time()
print(f"Time taken(seconds) : {t2_t-t1_t}")

Download process name started : Download with thread 1
Download process name started : Download with thread 2
Download process name started : Download with thread 3
Main program done!!
Time taken(seconds) : 0.007508516311645508


### thread.join()

In [19]:
t1_t = time.time()
t1.start()
t2.start()
t3.start()

t1.join()
t2.join()
t3.join() # next line will be executed once thread t3 will get complete

print("Main program done!!")
t2_t = time.time()
print(f"Time taken(seconds) : {t2_t-t1_t}")

Download process name started : Download with thread 1
Download process name started : Download with thread 2
Download process name started : Download with thread 3
File downloaded successfully
Process name completed : Download with thread 1
File downloaded successfully
Process name completed : Download with thread 3
File downloaded successfully
Process name completed : Download with thread 2
Main program done!!
Time taken(seconds) : 0.3011481761932373


#Closing Tread
with thread.join we are not closing the thread, we are making the main thread wait until the thread completes

In [1]:
import requests
import time
import threading
import time
import os

In [2]:
url = "https://raw.githubusercontent.com/Monalsingh/VideoBroadcaster/refs/heads/main/static/default-office-animated.png"

def download_file(process_name, url, file_path):
    try:
        print(f"Download process name started : {process_name}")
        response = requests.get(url)
        with open(file_path, "wb") as file:
            for chunk in response.iter_content(chunk_size=8192):
                if chunk:
                    file.write(chunk)
        print("File downloaded successfully")
    except Exception as e :
        print(f"Error downloading file : {e}")
    print(f"Process name completed : {process_name}")


In [5]:
# daemon=True - Marks the thread as a daemon thread, meaning it will automatically stop when the main program exits

In [3]:
t1_wd = threading.Thread(target=download_file, args=("abc_1", url, "a.png"), daemon=True) # daemon

t1_wd.start()
t1_wd.join()

print("Process completed")

Download process name started : abc_1
File downloaded successfully
Process name completed : abc_1
Process completed


# Close Manually - threading.Event()

use the threading.Event() object to gracefully control and stop a thread from the main program.

In [7]:
stop_signal = threading.Event() # Creates an Event object (default state = False)

def watch_file(file_path):
    count = 0
    print(f"Watching {file_path}")
    while not stop_signal.is_set(): # Keep running while the flag is False - .is_set() → returns True if the flag is set; otherwise False.
        print("Watching file ....")
        time.sleep(2)
    print("Watch exiting cleanly..")


t1_wd = threading.Thread(target=watch_file, args=("a.txt",))
t1_wd.start()

stop_signal.set() # When you call stop_signal.set(), it turns True, signaling the thread to stop.
# ^ told the thread to stop

t1_wd.join()  # Wait for it to stop

time.sleep(5)

print("Main program exited cleanly.")

Watching a.txt
Watching file ....
Watch exiting cleanly..
Main program exited cleanly.


## cooperative method where the thread voluntarily checks a flag and decides to stop.



In [8]:

stop_event = {"thread_1": False, "thread_2": False, "thread_3": False}

def watch_file(thread_id, file_path, thread_name):
    print(f"Watching {file_path}")
    while not stop_event[thread_name]:
        print("Watching file ....")
        time.sleep(2)
    print(f"Watch thread id : {thread_id } exiting cleanly..")


t1_wd = threading.Thread(target=watch_file, args=(1 ,"a.txt","thread_1",))
t2_wd = threading.Thread(target=watch_file, args=(2 ,"b.txt","thread_2",))
t3_wd = threading.Thread(target=watch_file, args=(3, "c.txt","thread_3",))

t1_wd.start()
t2_wd.start()
t3_wd.start()


time.sleep(5)
stop_event["thread_1"] = True # Event value to be true

time.sleep(5)
stop_event["thread_2"] = True # Event value to be true

time.sleep(5)
stop_event["thread_3"] = True # Event value to be true


t1_wd.join()
t2_wd.join()
t3_wd.join()

time.sleep(5)

print("Main program exited cleanly.")

Watching a.txt
Watching file ....
Watching b.txt
Watching file ....
Watching c.txt
Watching file ....
Watching file ....
Watching file ....
Watching file ....
Watching file ....
Watching file ....
Watching file ....
Watch thread id : 1 exiting cleanly..
Watching file ....
Watching file ....
Watching file ....Watching file ....

Watch thread id : 2 exiting cleanly..Watching file ....

Watching file ....
Watching file ....
Watch thread id : 3 exiting cleanly..
Main program exited cleanly.


##refined and safer version of earlier program that uses a dict of flags to signal thread termination, but now it includes a threading.Lock() to ensure thread-safety.

Why use a lock here ?

Because multiple threads share the same dictionary (stop_event), and Python’s built-in data structures are not guaranteed to be thread-safe.

Without a lock, simultaneous read/write access could result in:

Corrupted data

Race conditions

Inconsistent behavior

In [13]:

stop_event = {"thread_1": False, "thread_2": False, "thread_3": False}
stop_event_lock = threading.Lock() # stop_event_lock: A mutex (Lock) to prevent race conditions when reading/writing stop_event.

def watch_file(thread_id, file_path, thread_name):
    print(f"Watching {file_path}")
    while True:
        with stop_event_lock: # Critical section
            if stop_event[thread_name]:
                break # Exit loop if stop flag is True
            print("Watching file ....")
        time.sleep(2) # placed outside the lock, which is good practice (locks shouldn't be held during long operations).

    print(f"Watch thread id : {thread_id } exiting cleanly..")


t1_wd = threading.Thread(target=watch_file, args=(1 ,"a.txt","thread_1"))
t2_wd = threading.Thread(target=watch_file, args=(2 ,"b.txt","thread_2"))
t3_wd = threading.Thread(target=watch_file, args=(3, "c.txt","thread_3"))

t1_wd.start()
t2_wd.start()
t3_wd.start()


time.sleep(5)
with stop_event_lock: # with stop_event_lock: block to ensure thread-safe updates.
    stop_event["thread_1"] = True # Event value to be true

time.sleep(5)
with stop_event_lock:
    stop_event["thread_2"] = True # Event value to be true

time.sleep(5)
with stop_event_lock:
    stop_event["thread_3"] = True# Event value to be true


t1_wd.join()
t2_wd.join()
t3_wd.join()

time.sleep(5)

print("Main program exited cleanly.")

Watching a.txt
Watching file ....
Watching b.txt
Watching file ....
Watching c.txt
Watching file ....
Watching file ....
Watching file ....
Watching file ....
Watching file ....
Watching file ....
Watching file ....
Watch thread id : 1 exiting cleanly..
Watching file ....
Watching file ....
Watching file ....
Watching file ....
Watching file ....
Watching file ....
Watch thread id : 2 exiting cleanly..
Watching file ....
Watching file ....
Watch thread id : 3 exiting cleanly..
Main program exited cleanly.


# ThreadPoolExecutor
example of using Python's high-level ThreadPoolExecutor from the concurrent.futures module to run multiple threads efficiently.

It handles .start(), .join() and queuing for you

In [14]:
from concurrent.futures import ThreadPoolExecutor

a = [(1 ,"a.txt","thread_1",), (2 ,"b.txt","thread_2",), (3 ,"c.txt","thread_3",), (4 ,"d.txt","thread_4",)]
def watch_file(input_tuple):
    thread_id, file_path, thread_name = input_tuple
    count=0
    print(f"Watching {file_path}")
    while count<5:
        print(f"Watching file ....{thread_id}")
        time.sleep(2)
        count+=1
    print(f"Watch thread id : {thread_id } exiting cleanly..")
    return f"Hello {thread_id}"


with ThreadPoolExecutor(max_workers=3) as executor:
    results = list(executor.map(watch_file, a))

# watch_file, (1 ,"a.txt","thread_1",)
# watch_file, (2 ,"b.txt","thread_2",)
# watch_file, (3 ,"c.txt","thread_3",)
# watch_file, (4 ,"d.txt","thread_4",)

print("Thread executed successfully")

Watching a.txt
Watching file ....1
Watching b.txt
Watching file ....2
Watching c.txt
Watching file ....3
Watching file ....1
Watching file ....2
Watching file ....3
Watching file ....1
Watching file ....2
Watching file ....3
Watching file ....1
Watching file ....2
Watching file ....3
Watching file ....1
Watching file ....2
Watching file ....3
Watch thread id : 1 exiting cleanly..
Watching d.txt
Watching file ....4
Watch thread id : 2 exiting cleanly..
Watch thread id : 3 exiting cleanly..
Watching file ....4
Watching file ....4
Watching file ....4
Watching file ....4
Watch thread id : 4 exiting cleanly..
Thread executed successfully


#MultiPocessing - (Parellel processing)

In [17]:
import multiprocessing
import time

def download_file(process_name, url, file_path):
    try:
        print(f"Download process name started : {process_name}")
        response = requests.get(url)
        with open(file_path, "wb") as file:
            for chunk in response.iter_content(chunk_size=8192):
                if chunk:
                    file.write(chunk)
        print("File downloaded successfully")
    except Exception as e :
        print(f"Error downloading file : {e}")
    print(f"Process name completed : {process_name}")

if __name__ == "__main__":
    url = "https://raw.githubusercontent.com/Monalsingh/VideoBroadcaster/refs/heads/main/static/default-office-animated.png"

    p1 = multiprocessing.Process(target=download_file, args=("Download with process 1", url, "a1.png"))
    p2 = multiprocessing.Process(target=download_file, args=("Download with process 2", url, "b1.png"))
    p3 = multiprocessing.Process(target=download_file, args=("Download with process 3", url, "c1.png"))

    start_time = time.time()

    p1.start()
    p2.start()
    p3.start()

    # Optional: wait for all processes to complete
    p1.join()
    p2.join()
    p3.join()

    end_time = time.time()

    print("Main program done!!")
    print(f"Time taken(seconds) : {end_time - start_time}")


Download process name started : Download with process 1
Download process name started : Download with process 2Download process name started : Download with process 3

File downloaded successfully
Process name completed : Download with process 1
File downloaded successfully
Process name completed : Download with process 2
File downloaded successfully
Process name completed : Download with process 3
Main program done!!
Time taken(seconds) : 1.1278061866760254


#Using multiprocessing pool
to Square Numbers in Parallel

In [15]:
import multiprocessing
import time

def square(n):
    print(f"Processing {n} in process {multiprocessing.current_process().name}")
    time.sleep(1)  # simulate heavy work
    return n * n

if __name__ == "__main__":
    numbers = [1, 2, 3, 4, 5]

    # Create a Pool of 3 worker processes
    with multiprocessing.Pool(processes=3) as pool:
        results = pool.map(square, numbers)

    print("Squares:", results)

Processing 1 in process ForkPoolWorker-1
Processing 2 in process ForkPoolWorker-2Processing 3 in process ForkPoolWorker-3

Processing 4 in process ForkPoolWorker-1
Processing 5 in process ForkPoolWorker-2
Squares: [1, 4, 9, 16, 25]


# Multiprocessing VS Threading

In [2]:
import time
from concurrent.futures import ThreadPoolExecutor
from multiprocessing import Pool

def cpu_task(power):
    count = 0
    for i in range(10**power):
        count += 1
    return count

if __name__ == "__main__":
    num_tasks = 15
    power = 8

    print("Comparing Multithreading vs. Multiprocessing (CPU-bound Task)")

    # Multithreading
    start_time_thread = time.time()
    with ThreadPoolExecutor(max_workers=num_tasks) as executor:
        results_thread = list(executor.map(cpu_task, [power]*num_tasks))
    end_time_thread = time.time()
    time_taken_thread = end_time_thread - start_time_thread
    print(f"Multithreading Time Taken: {time_taken_thread:.4f} seconds")
    print(f"Thread Results (first few): {results_thread[:2]}\n")

    # Multiprocessing
    start_time_process = time.time()
    with Pool(processes=num_tasks) as pool:
        results_process = list(pool.map(cpu_task, [power]*num_tasks))
    end_time_process = time.time()
    time_taken_process = end_time_process - start_time_process
    print(f"Multiprocessing Time Taken: {time_taken_process:.4f} seconds")
    print(f"Process Results (first few): {results_process[:2]}\n")

    print("Comparison:")
    if time_taken_process < time_taken_thread:
        speedup = time_taken_thread / time_taken_process
        print(f"Multiprocessing was significantly faster, achieving a speedup of {speedup:.2f}x.")
    elif time_taken_thread < time_taken_process:
        slowdown = time_taken_process / time_taken_thread
        print(f"Multiprocessing was slower by a factor of {slowdown:.2f}x.")
    else:
        print("Multithreading and Multiprocessing performance was similar.")

Comparing Multithreading vs. Multiprocessing (CPU-bound Task)
Multithreading Time Taken: 66.8532 seconds
Thread Results (first few): [100000000, 100000000]

Multiprocessing Time Taken: 70.4403 seconds
Process Results (first few): [100000000, 100000000]

Comparison:
Multiprocessing was slower by a factor of 1.05x.


NameError: name 'add' is not defined